In [11]:
# transfermarkt 사이트에서 이적료가 높은 선수들의 성과 분석 및 이적료에 미치는 중요 변수 분석
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

# 헤더 정보를 넣어주지 않으면 403 에러가 발생할 수 있다.
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}

# 파싱 편의를 위해 사용자 정의 함수 만들어 놓기
def parsingSite(url):
    response=requests.get(url, headers=headers)
    response.status_code
    time.sleep(1)
    # BeautifulSoup으로 웹페이지 분석 준비하기
    psdSite=BeautifulSoup(response.text, 'html.parser')
    return psdSite

# 필요한 정보를 담을 리스트 생성
rank=[] # 순위
player=[] # 선수정보
name=[] # 이름
position=[] # 포지션
age=[] # 나이
country=[] # 국적
club=[] # 소속팀
value=[] # 이적료
matchNum=[] # 경기수 
goal=[] # 골
assist=[] # 어시스트
yellowCard=[] # 옐로카드
redCard=[] # 레드카드
subIn=[] # 교체투입
subOut=[] # 교체아웃
player_list=[] # 선수 정보를 담을 리스트

# 20페이지(500명) 크롤링해보기 #추후 포지션별로 나눠야할듯 데이터가 너무 적음
for a in range(1,5):
    for i in range(1, 21):
        # 나중에 한국사이트가 아닌 외국사이트로 바꿔야할 수도 있음 .co.kr -> .com 한국사이트는 한글과 영어가 혼합되어있는 경우가 많아서 외국사이트로 변경경
        if a==1:
            url=f'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?land_id=0&ausrichtung=Torwart&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={i}'
        elif a==2:
            url=f'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?land_id=0&ausrichtung=Abwehr&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={i}'
        elif a==3:
            url=f'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?land_id=0&ausrichtung=Mittelfeld&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={i}'
        else:
            url=f'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop?land_id=0&ausrichtung=Sturm&spielerposition_id=alle&altersklasse=alle&jahrgang=0&kontinent_id=0&plus=1&page={i}'
        site1=parsingSite(url)
        
        # 선수 정보가 담긴 태그와 클래스 찾기
        player_info = site1.find_all('tr', class_=['odd', 'even'])
        #print(player_info)
        #print(len(player_info))

        # 가져온 정보 중에서 필요한 정보만 추출하기
        for info in player_info:
            player = info.find_all('td')
            rank = player[0].text
            name = player[3].text
            position = player[4].text
            age = player[5].text
            country = player[6].img['alt']
            club = player[7].img['alt']
            value = player[8].text.strip() # value=value[1:-1] 아예 숫자만 가져오기
            matchNum = player[9].text
            goal = player[10].text
            assist = player[12].text
            yellowCard = player[13].text
            redCard = player[15].text
            subIn = player[16].text
            subOut = player[17].text

            player_list.append([rank, name, position, age, country, club, value, matchNum, goal, assist, yellowCard, redCard, subIn, subOut])
        
        time.sleep(1)
#print(player_list)
# 추출한 정보를 csv 파일로 저장하기
df=pd.DataFrame(player_list, columns=['rank', 'name', 'position', 'age', 'country', 'club', 'value', 'matchNum', 'goal', 'assist', 'yellowCard', 'redCard', 'subIn', 'subOut'])
#df.to_csv('player_info.csv', index=False)
#print(df.head())

# 결측치 처리, 결측치는 없지만 혹시 모르니 해놓기
df.fillna(0, inplace=True)

# 데이터 정제(타입변경 및 이적료 유로->원화환산 ex)30000000.0 ->3000억원)
for c in range(0,10):
    if df['value'].str.contains(f'.{c}0m').any() == 1:
        df['value']=df['value'].str.replace(f'.{c}0m', f'{c}00000')
df['value']=df['value'].str.replace('€', '').astype(float)*(0.15)
df['age'] = df['age'].astype(int)
df['matchNum'] = df['matchNum'].astype(int)
df['goal'] = df['goal'].astype(int)
df['assist'] = df['assist'].astype(int)
df['yellowCard'] = df['yellowCard'].astype(int)
df['redCard'] = df['redCard'].astype(int)
df['subIn'] = df['subIn'].astype(int)
df['subOut'] = df['subOut'].astype(int)

# 경기당 득점, 어시스트, 카드 수 등 파생 변수 생성
'''
df['goalPerMatch'] = df['goal']/df['matchNum']
df['assistPerMatch'] = df['assist']/df['matchNum']
df['yellowCardPerMatch'] = df['yellowCard']/df['matchNum']
df['redCardPerMatch'] = df['redCard']/df['matchNum']
'''

#print(df.head())

df['name'].count()

#기초 통계 분석
#print(df.describe())

np.int64(2000)